In [2]:
import streamlit as st
import pandas as pd
import numpy as np
import pickle
from SpotifyRecommender import SpotifyRecommender
import difflib
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# SPOTIFY API
client_id = "7dd1627b4bcd44d494ca70d954e02eaf"
client_secret = "55fa312e777e49669556c038ab11906e"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [11]:
test_track_id = "54bm2e3tk8cliUz3VSdCPZ"
features = sp.audio_features(test_track_id)
# analysis = sp.audio_analysis(test_track_id)
features

[{'danceability': 0.539,
  'energy': 0.828,
  'key': 4,
  'loudness': -9.108,
  'mode': 0,
  'speechiness': 0.035,
  'acousticness': 0.00067,
  'instrumentalness': 0.421,
  'liveness': 0.154,
  'valence': 0.562,
  'tempo': 105.25,
  'type': 'audio_features',
  'id': '54bm2e3tk8cliUz3VSdCPZ',
  'uri': 'spotify:track:54bm2e3tk8cliUz3VSdCPZ',
  'track_href': 'https://api.spotify.com/v1/tracks/54bm2e3tk8cliUz3VSdCPZ',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/54bm2e3tk8cliUz3VSdCPZ',
  'duration_ms': 515387,
  'time_signature': 4}]

In [15]:
track_info = sp.track(test_track_id)
track_info

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2ye2Wgw4gimLv2eAKyk1NB'},
    'href': 'https://api.spotify.com/v1/artists/2ye2Wgw4gimLv2eAKyk1NB',
    'id': '2ye2Wgw4gimLv2eAKyk1NB',
    'name': 'Metallica',
    'type': 'artist',
    'uri': 'spotify:artist:2ye2Wgw4gimLv2eAKyk1NB'}],
  'available_markets': ['CA', 'US'],
  'external_urls': {'spotify': 'https://open.spotify.com/album/7CGhx630DIjdJqaBDVKc5j'},
  'href': 'https://api.spotify.com/v1/albums/7CGhx630DIjdJqaBDVKc5j',
  'id': '7CGhx630DIjdJqaBDVKc5j',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b273cad4832cb7b5844343278daa',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e02cad4832cb7b5844343278daa',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d00004851cad4832cb7b5844343278daa',
    'width': 64}],
  'name': 'Master of Puppets (Remastered Deluxe Box Set)',
  'r

In [17]:
artist_id = track_info['artists'][0]['id']
artist_info = sp.artist(artist_id)
features[0]['artists_genres'] = artist_info['genres']

In [18]:
data_for_prediction = pd.DataFrame(features)
data_for_prediction

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artists_genres
0,0.539,0.828,4,-9.108,0,0.035,0.00067,0.421,0.154,0.562,105.25,audio_features,54bm2e3tk8cliUz3VSdCPZ,spotify:track:54bm2e3tk8cliUz3VSdCPZ,https://api.spotify.com/v1/tracks/54bm2e3tk8cl...,https://api.spotify.com/v1/audio-analysis/54bm...,515387,4,"[hard rock, metal, old school thrash, rock, th..."


In [20]:
model=SpotifyRecommender()
model.load('SpotifyRecommender_apiFeatures.pkl')

In [27]:
output = model.predict(data_for_prediction,k=300,match_genre=True)
output.head(10)

,name,artists_name,artists_genres,album_name,track_href,preview_url,analysis_url,href,lyrics,playlist,...,artists_followers,artists_artist_popularity,artists_id,album_release_date,album_images,album_total_tracks,album_external_urls,album_id,distances,match_genre_count
2423,Enter Sandman,Metallica,"[hard rock, metal, old school thrash, rock,...",Metallica,https://api.spotify.com/v1/tracks/5BIMPccDwShp...,https://p.scdn.co/mp3-preview/1ee9f1b0c3e3b327...,https://api.spotify.com/v1/audio-analysis/5BIM...,https://api.spotify.com/v1/tracks/5BIMPccDwShp...,"\r\n\r\nSay your prayers, little one\r\nDon't ...",Ultimate Hockey,...,9884031,83,2ye2Wgw4gimLv2eAKyk1NB,1991-08-12,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",12,{'spotify': 'https://open.spotify.com/album/6Q...,6QdCohkHKNTVoaSx1ZzitH,1.292634,6
52489,Walk - Remastered,Pantera,"[alternative metal, groove metal, hard rock,...",The Best Of Pantera: Far Beyond The Great Sout...,https://api.spotify.com/v1/tracks/0wcKfCfSBaG0...,https://p.scdn.co/mp3-preview/9a50681d7035b1e7...,https://api.spotify.com/v1/audio-analysis/0wcK...,https://api.spotify.com/v1/tracks/0wcKfCfSBaG0...,\r\n\r\nWell it's floodin' down in Texas\r\nAl...,I Love My Air Guitar,...,1940609,70,14pVkFUHDL207LzLHtSA18,2003-09-23,"[{'height': 567, 'url': 'https://i.scdn.co/ima...",16,{'spotify': 'https://open.spotify.com/album/2p...,2pEiQl69yF4PZDXIBoZjaH,1.365172,6
74490,Balls to the Wall,Accept,"[album rock, german metal, glam metal, hard...",Balls To The Wall,https://api.spotify.com/v1/tracks/28sMmBCmfHoy...,https://p.scdn.co/mp3-preview/f17ccce8323b75f9...,https://api.spotify.com/v1/audio-analysis/28sM...,https://api.spotify.com/v1/tracks/28sMmBCmfHoy...,\r\nToo many slaves in this world\r\nDie by to...,Heavy Metal,...,321460,56,3JDIAtVrJdQ7GFOX26LYpv,1983-01-01,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",12,{'spotify': 'https://open.spotify.com/album/2o...,2ogbkPbabA9E5LFXiFhDas,1.383410,4
63846,(Here Comes The) Rain,The Cult,"[album rock, classic rock, dance rock, glam...",Love (Expanded Edition),https://api.spotify.com/v1/tracks/2avqFwSrY5t2...,https://p.scdn.co/mp3-preview/1d0007aa9e299a00...,https://api.spotify.com/v1/audio-analysis/2avq...,https://api.spotify.com/v1/tracks/2avqFwSrY5t2...,-99,Classic Rock Workout,...,244945,59,49DW3KvkyjHO35mK1JnSyS,1985-01-01,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",21,{'spotify': 'https://open.spotify.com/album/2w...,2w7mhWYo5yEEVqYIt7tdf1,1.109166,3
53458,Please Don't Leave Me,Pretty Maids,"[album rock, danish metal, glam metal, hard...",Sin-Decade,https://api.spotify.com/v1/tracks/77ZIkT6DwOXP...,https://p.scdn.co/mp3-preview/769ea10f06df0dc5...,https://api.spotify.com/v1/audio-analysis/77ZI...,https://api.spotify.com/v1/tracks/77ZIkT6DwOXP...,\r\n(Ooooh)\r\nIn the summer we'd be crazy\r\n...,Pop/Rock klassikere,...,35103,45,4OGo8f0tdKloP3IZwvnLqt,1992-01-01,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",11,{'spotify': 'https://open.spotify.com/album/6F...,6F6n2HxXHoKgEvZQltIiB2,1.268451,3
31700,When It's Love,Van Halen,"[album rock, classic rock, hard rock, metal...",OU812,https://api.spotify.com/v1/tracks/2rxePraWVAXy...,https://p.scdn.co/mp3-preview/0bce03604295b8f2...,https://api.spotify.com/v1/audio-analysis/2rxe...,https://api.spotify.com/v1/tracks/2rxePraWVAXy...,\r\n\r\nEverybody's looking for something\r\nS...,Power Ballads,...,2260401,74,2cnMpRsOVqtPMfq7YiFE6K,1988-05-15,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",10,{'spotify': 'https://open.spotify.com/album/1r...,1rnKiZALdvi91F0fepxBUe,1.278737,3
12813,Sympathy For The Devil,Motörhead,"[album rock, alternative metal, classic rock...",Under Cöver,https://api.spotify.com/v1/tracks/0gNfYuNUfuMi...,https://p.scdn.co/mp3-preview/f45550b6e8ca9c39...,https://api.spotify.com/v1/audio-analysis/0gNf...,https://api.spotify.com/v1/tracks/0gNfYuNUfuMi...,\r\n\r\nPlease allow me to introduce myself\r\...,Saatanan hyvää musiikkia,...,1616578,68,1DFr97A9HnbV3SKTJFu62M,2017-09-01,"[{'hei